In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [17]:
def get_mapper(X, y, n_categories, columns=None):
    '''
    ----
    n_categories: int or dict
    '''
    class Map:
        def __init__(self, mapping):
            self.mapping = pd.Series(mapping, dtype='Int16', name='codes')
            if pd.isna(self.mapping.index).sum() == 0:
                self.na_cat_code = self.mapping.median() - 1
                self.mapping = self.mapping.append(pd.Series({np.nan:self.na_cat_code}))
            else:
                self.na_cat_code = self.mapping[pd.isna(self.mapping.index)].values[0]
        def __str__(self):
            return 'Map:\nColumn Value -> Category Code\n' + str(self.mapping)
        def __repr__(self):
            return 'Map:\nColumn Value -> Category Code\n' + repr(self.mapping)
        def __call__(self, key):            
            return self[key]
        def __getitem__(self, key):
            try:
                return self.mapping[key]
            except:
                return self.na_cat_code

    from pandas.api.types import is_numeric_dtype    
    columns = X.columns if columns is None else columns
    y = y.copy(deep=True)
    df = X.copy(deep=True).join(y, how='inner')
    y_col = y.name    
    mapper = {}
    for col in columns:
        nc = n_categories[col] if type(n_categories) is dict else n_categories
        assert (nc > 1), 'n_categories must be greater than 1'
        if is_numeric_dtype(df[col]) and df[col].nunique() > nc:
            # covert to ordinal
            values = df[col].tolist() + [float('-inf'), float('inf')]            
            cats = pd.qcut(values, nc, duplicates='drop').dtype.categories            
            df[col] = pd.cut(df[col], bins=cats, duplicates='drop')
            
        # remove extra categories
        vcounts = df[col].value_counts(dropna=False)
        if vcounts.size > nc:
            keep, setna = set(vcounts.iloc[:nc].index), set(vcounts.iloc[nc:].index)
            mp = {x: x for x in keep}
            mp.update({x: np.nan for x in setna})
            df[col] = df[col].map(mp)

        sum_ser = df.groupby(col, dropna=False)[y_col].agg(sum)
        sum_ser.name = 'sum'
        count_ser = df.groupby(col, dropna=False)[y_col].count()
        count_ser.name = 'count'
        stat_df = pd.merge(count_ser, sum_ser, right_index=True, left_index=True)
        stat_df['ratio'] = stat_df['sum'] / stat_df['count']
        stat_df = stat_df.sort_values(by=['ratio', 'count', col])
                                        
        mp = [None] * len(stat_df.index)
        mp[0] = stat_df.iloc[0]['count']
        for i in range(1, len(mp)):            
            mp[i] = mp[i-1] + stat_df.iloc[i]['count']
        mapper[col] = Map({v:mp[i] for i, v in enumerate(stat_df.index)})
#         mapper[col] = Map({v:i*1.0 for i, v in enumerate(stat_df.index)})        
    return mapper

In [18]:
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
print(f'train dataset size: {train_df.shape}')
print(f'test dataset size: {test_df.shape}')
print('train columns:')
print(train_df.columns.tolist())
print('test columns:')
print(test_df.columns.tolist())
print('train specific column(s):')
print(set(train_df.columns) - set(test_df.columns))

train dataset size: (891, 12)
test dataset size: (418, 11)
train columns:
['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
test columns:
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
train specific column(s):
{'Survived'}


In [19]:
def expand_names(df):    
    df_exp = df.Name.str.split(', ', n=1, expand=True)
    df['Lastname'] = df_exp[0]
    df_exp = df_exp[1].str.split('.', n=1, expand=True)    
    df['Title'] = df_exp[0]    

def expand_ticket(df):    
    s = (df['Ticket'].str.split(expand=True))[0]    
    s[s.str.isdigit()] = np.nan
    df['Ticket_grp'] = s

expand_names(train_df)
expand_names(test_df)
expand_ticket(train_df)
expand_ticket(test_df)
# fare_impute, age_impute = train_df['Fare'].mean(), train_df['Age'].mean()
# train_df['Fare'].fillna(fare_impute, inplace=True)
# test_df['Fare'].fillna(fare_impute, inplace=True)
# train_df['Age'].fillna(age_impute, inplace=True)
# test_df['Age'].fillna(age_impute, inplace=True)
# train_df['FareBin'] = pd.qcut(train_df['Fare'], 7)
# train_df['AgeBin'] = pd.qcut(train_df['Age'].astype(int), 7, duplicates='drop')
# test_df['FareBin'] = test_df['Fare'].astype(train_df['FareBin'].dtype)
# test_df['AgeBin'] = test_df['Age'].astype(train_df['AgeBin'].dtype)

# cols_mask = set(['Survived', 'PassengerId', 'Name', 'Ticket', 'Age', 'Fare'])

In [20]:
train_df.head(3)

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   

   Parch            Ticket     Fare Cabin Embarked   Lastname Title Ticket_grp  
0      0         A/5 21171   7.2500   NaN        S     Braund    Mr        A/5  
1      0          PC 17599  71.2833   C85        C    Cumings   Mrs         PC  
2      0  STON/O2. 3101282   7.9250   NaN        S  Heikkinen  Miss   STON/O2.

In [21]:
test_df.sample(3)

PassengerId  Pclass                  Name   Sex   Age  SibSp  Parch  \
121         1013       3     Kiernan, Mr. John  male   NaN      1      0   
83           975       3  Demetri, Mr. Marinko  male   NaN      0      0   
81           973       1    Straus, Mr. Isidor  male  67.0      1      0   

       Ticket      Fare    Cabin Embarked Lastname Title Ticket_grp  
121    367227    7.7500      NaN        Q  Kiernan    Mr        NaN  
83     349238    7.8958      NaN        S  Demetri    Mr        NaN  
81   PC 17483  221.7792  C55 C57        S   Straus    Mr         PC

In [65]:
class NaiveClassifier:
    def __init__(self):        
        self.model = None
    
    def fit(self, X, y):         
        self.model = None
        max_acc = 0.0
        for col in X:
            pcat, tp1 = list(X.loc[y==1, col].value_counts().items())[0]
            tn1 = X[(X[col] != pcat) & (y==0)].shape[0]
            ncat, tn2 = list(X.loc[y==0, col].value_counts().items())[0]
            tp2 = X[(X[col] != ncat) & (y==1)].shape[0]
            plan = (1, col, pcat) if tp1 + tn1 >= tp2 + tn2 else (0, col, ncat)
            acc = (1/X.shape[0]) * (X[(y==plan[0]) & (X[plan[1]] == plan[2])].shape[0] + 
                                    X[(y==(1-plan[0])) & (X[plan[1]] != plan[2])].shape[0])
            if acc > max_acc:
                self.model = plan
                max_acc = acc
                max_col = col
                
    def predict(self, X):
        assert(self.model != None), 'Classifier is not trained'
        predictions = pd.Series(0, name='predictions', index=X.index)
        label = self.model[0]
        col = self.model[1]
        val = self.model[2]
        predictions[X[col] == val] = label
        predictions[X[col] != val] = 1-label
        return predictions
    
    def __str__(self):
        return f'all X[{self.model[1]}] == {self.model[2]} are {self.model[0]} otherwise {1-self.model[0]}'

In [75]:
%%time

from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

features = ['Sex', 'Age', 'SibSp', 'Parch', 'Ticket_grp', 'Fare', 'Cabin', 'Embarked', 'Title']
# n_categories = {'Sex': 2, 'Age': 10, 'SibSp': 7, 'Parch': 7, 'Ticket_grp':10, 'Fare':7, 'Cabin':4, 'Embarked':3, 
#                 'Lastname':10, 'Title':4}

y = train_df['Survived'].copy()
X = train_df[features].copy()

models = {'naive': NaiveClassifier(), 
          'nbayes': GaussianNB(),
          'rf': RandomForestClassifier(), 
          'xgb': XGBClassifier(),
          'svc': make_pipeline(StandardScaler(), SVC()),
          'lr': make_pipeline(StandardScaler(), LogisticRegression()), 
          'knn': make_pipeline(StandardScaler(), KNeighborsClassifier())}

n_splits = 50
n_cat_range = [2,3,4,8,16,32]
result_cols = (['test_baseline', 'train_baseline', 'test_naive', 'train_naive'] + 
               [f'{grp}_{m}_{n_cat}' for m in models for n_cat in n_cat_range for grp in ['test', 'train']])
results = pd.DataFrame(columns = result_cols, index = range(n_splits), dtype='float64')
for i in range(n_splits):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.47)
    results.loc[i, 'train_baseline'] = max(y_train.mean(), 1- y_train.mean())
    results.loc[i, 'test_baseline'] = max(y_valid.mean(), 1- y_valid.mean())
    
    model = NaiveClassifier()
    model.fit(X_train, y_train)    
    print(model)
    predictions_train = model.predict(X_train)
    results.loc[i, 'train_naive'] = accuracy_score(predictions_train, y_train)
    predictions_test = model.predict(X_valid)
    results.loc[i, 'test_naive'] = accuracy_score(predictions_test, y_valid)
    
    Xt = X_train.copy()
    Xv = X_valid.copy()
    for n_categories in n_cat_range:        
        mapper = get_mapper(X_train, y_train, columns=features, n_categories=n_categories)
        for col in features:
            Xt[col] = X_train[col].map(mapper[col])
            Xv[col] = X_valid[col].map(mapper[col])
            
        for name, model in models.items():                        
            model.fit(Xt, y_train)            
            pred_train = model.predict(Xt)
            score_train = accuracy_score(pred_train, y_train)            
            results.loc[i, f'train_{name}_{n_categories}'] = score_train
            pred_test = model.predict(Xv)
            score_test = accuracy_score(pred_test, y_valid)
            results.loc[i, f'test_{name}_{n_categories}'] = score_test

all X[Sex] == female are 1 otherwise 0
all X[Title] == Mr are 0 otherwise 1
all X[Sex] == female are 1 otherwise 0
all X[Title] == Mr are 0 otherwise 1
all X[Title] == Mr are 0 otherwise 1
all X[Sex] == female are 1 otherwise 0
all X[Title] == Mr are 0 otherwise 1
all X[Sex] == female are 1 otherwise 0
all X[Title] == Mr are 0 otherwise 1
all X[Sex] == female are 1 otherwise 0
all X[Sex] == female are 1 otherwise 0
all X[Title] == Mr are 0 otherwise 1
all X[Title] == Mr are 0 otherwise 1
all X[Sex] == female are 1 otherwise 0
all X[Sex] == female are 1 otherwise 0
all X[Sex] == female are 1 otherwise 0
all X[Sex] == female are 1 otherwise 0
all X[Sex] == female are 1 otherwise 0
all X[Sex] == female are 1 otherwise 0
all X[Sex] == female are 1 otherwise 0
all X[Sex] == female are 1 otherwise 0
all X[Sex] == female are 1 otherwise 0
all X[Title] == Mr are 0 otherwise 1
all X[Sex] == female are 1 otherwise 0
all X[Sex] == female are 1 otherwise 0
all X[Sex] == female are 1 otherwise 0
al

In [78]:
cols_train = results.columns[results.columns.str.contains('train')]
cols_test = results.columns[results.columns.str.contains('test')]
results[cols_test].apply(lambda x: x- results['test_naive']).describe().T.sort_values(by='min', ascending=False)

count      mean       std       min       25%       50%  \
test_svc_16      50.0  0.014940  0.016484 -0.035800  0.004773  0.016706   
test_svc_4       50.0  0.013031  0.012750 -0.011933  0.004773  0.014320   
test_xgb_16      50.0 -0.001289  0.020247 -0.042959 -0.014320  0.000000   
test_svc_8       50.0  0.015418  0.014840 -0.026253  0.007757  0.015513   
test_xgb_4       50.0 -0.000477  0.019111 -0.040573 -0.013723 -0.002387   
test_svc_32      50.0  0.006635  0.016933 -0.042959 -0.002387  0.008353   
test_xgb_8       50.0 -0.001480  0.019591 -0.038186 -0.011933 -0.007160   
test_rf_8        50.0  0.005155  0.018395 -0.028640 -0.011337  0.007160   
test_rf_32       50.0  0.001623  0.018031 -0.045346 -0.008950  0.001193   
test_xgb_3       50.0  0.001909  0.018474 -0.035800 -0.009547  0.002387   
test_rf_4        50.0  0.003055  0.018537 -0.054893 -0.008950  0.003580   
test_nbayes_8    50.0 -0.059570  0.104848 -0.343675 -0.124105 -0.005967   
test_svc_3       50.0  0.005394  0.012918 -0.019093 -0.002387  0.002387   
test_lr_4        50.0 -0.005203  0.016610 -0.035800 -0.020883 -0.002387   
test_nbayes_16   50.0 -0.044105  0.054337 -0.205251 -0.083532 -0.038186   
test_rf_16       50.0  0.002005  0.016531 -0.035800 -0.007160  0.002387   
test_xgb_32      50.0 -0.005012  0.018874 -0.059666 -0.016706 -0.004773   
test_knn_16      50.0  0.001671  0.017639 -0.052506 -0.007160  0.002387   
test_knn_8       50.0 -0.006110  0.020796 -0.071599 -0.016110 -0.005967   
test_knn_3       50.0 -0.007064  0.018392 -0.066826 -0.015513 -0.004773   
test_rf_3        50.0  0.000764  0.014951 -0.040573 -0.009547  0.002387   
test_knn_4       50.0 -0.009165  0.016154 -0.040573 -0.020883 -0.011933   
test_lr_8        50.0  0.000573  0.015539 -0.038186 -0.008950  0.004773   
test_svc_2       50.0  0.002100  0.010981 -0.019093 -0.004773  0.002387   
test_lr_16       50.0 -0.000716  0.014068 -0.033413 -0.009547  0.001193   
test_lr_32       50.0 -0.007112  0.016810 -0.040573 -0.016706 -0.004773   
test_rf_2        50.0 -0.009212  0.019491 -0.050119 -0.021480 -0.005967   
test_knn_32      50.0 -0.007733  0.019494 -0.045346 -0.021480 -0.005967   
test_xgb_2       50.0 -0.008305  0.017520 -0.057279 -0.021480 -0.007160   
test_lr_3        50.0 -0.005442  0.013293 -0.047733 -0.011933 -0.004773   
test_nbayes_4    50.0 -0.053842  0.113069 -0.417661 -0.036993 -0.008353   
test_nbayes_32   50.0 -0.064582  0.047629 -0.171838 -0.092482 -0.059666   
test_lr_2        50.0 -0.001909  0.010057 -0.026253 -0.008950  0.000000   
test_knn_2       50.0 -0.015847  0.027082 -0.128878 -0.023866 -0.008353   
test_nbayes_3    50.0 -0.042768  0.080491 -0.408115 -0.031026 -0.016706   
test_nbayes_2    50.0 -0.056754  0.066280 -0.377088 -0.099642 -0.028640   
test_naive       50.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_naive_32    50.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_naive_16    50.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_naive_8     50.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_naive_4     50.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_naive_3     50.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_naive_2     50.0  0.000000  0.000000  0.000000  0.000000  0.000000   
test_baseline    50.0 -0.164726  0.023061 -0.221957 -0.178998 -0.163484   

                     75%       max  
test_svc_16     0.025656  0.052506  
test_svc_4      0.019093  0.050119  
test_xgb_16     0.009547  0.050119  
test_svc_8      0.025656  0.045346  
test_xgb_4      0.011337  0.045346  
test_svc_32     0.018496  0.042959  
test_xgb_8      0.008950  0.042959  
test_rf_8       0.018496  0.042959  
test_rf_32      0.011933  0.040573  
test_xgb_3      0.013723  0.040573  
test_rf_4       0.016706  0.038186  
test_nbayes_8   0.009547  0.038186  
test_svc_3      0.013723  0.038186  
test_lr_4       0.004773  0.035800  
test_nbayes_16  0.002387  0.035800  
test_rf_16      0.014320  0.035800  
test_xgb_32

In [26]:
cols_train = results.columns[results.columns.str.contains('train')]
cols_test = results.columns[results.columns.str.contains('test')]
topcols = sorted(set(results[cols_test].T.idxmax().values))
results[topcols].apply(lambda x: x - results['naive_test']).describe()

rf_128_test  rf_32_test  rf_8_test  svc_128_test  svc_8_test  \
count    10.000000   10.000000  10.000000  1.000000e+01   10.000000   
mean      0.025140    0.029609   0.021788  4.469274e-03    0.034637   
std       0.025702    0.028614   0.033875  1.931665e-02    0.026177   
min      -0.033520   -0.022346  -0.027933 -2.234637e-02   -0.005587   
25%       0.018156    0.013966  -0.006983 -1.117318e-02    0.019553   
50%       0.030726    0.036313   0.027933  5.551115e-17    0.039106   
75%       0.037709    0.047486   0.036313  2.234637e-02    0.054469   
max       0.061453    0.067039   0.083799  3.351955e-02    0.067039   

       xgb_32_test  xgb_8_test  
count    10.000000   10.000000  
mean      0.024022    0.022905  
std       0.026863    0.025458  
min      -0.011173   -0.022346  
25%       0.011173    0.001397  
50%       0.022346    0.030726  
75%       0.039106    0.043296  
max       0.078212    0.055866

In [ ]:
# Using cross validation

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, ShuffleSplit, GridSearchCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

features = ['Sex', 'Age', 'SibSp', 'Parch', 'Ticket_grp', 'Fare', 'Cabin', 'Embarked', 'Lastname', 'Title']

y_train = train_df['Survived'].copy()
X_train = train_df[features].copy()
X_test = test_df[features].copy()

my_mapping = get_mapper(X_train, y_train, n_categories=n_categories, columns=features)

for col in features:
    X_train[col] = X_train[col].map(my_mapping[col])
    X_test[col] = X_test[col].map(my_mapping[col])

# my_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), 
#                             RandomForestClassifier(criterion='gini', n_estimators=300, random_state=0, max_depth=8))
my_pipeline = RandomForestClassifier(criterion='gini', n_estimators=300, random_state=0, max_depth=8)
cv = ShuffleSplit(n_splits=40, test_size=.46)
scores = cross_val_score(my_pipeline, X_train, y_train, cv=cv, scoring='accuracy')
print(f'Model accuracy: {scores.mean()}')

In [ ]:
my_pipeline.fit(X_train, y_train)
predictions = my_pipeline.predict(X_test)
output_df = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': predictions})
# output_df.to_csv('my_submission_4.csv', index=False)